# <center><span style="color:DarkBlue">Modeling</span></center> <a class="tocSkip">

## 변수 설명
\**일차적으로 다음의 변수들은 모델링에 사용하지 않기로 함.*

- ~LoanNr_ChkDgt : 식별 키~
- ~Name : 차용 기업명(Borrower)~
- ~City : 차용 기업이 위치한 도시~
- State : 차용 기업이 위치한 주
- ~Zip : 차용 기업의 우편 번호~
- ~Bank : 은행 이름~
- ~BankState : 은행이 위치한 주~
- NAICS : 북미 산업 분류 체계 코드
- ~ApprovalDate : SBA 약정 발행 날짜~
- ~ApprovalFY : 약정 회계 연도~
- ~Term : 대출 기간(개월)~
- NoEmp : 직원 수
- NewExist : 0 = 기존 비즈니스, 1 = 신규 비즈니스
- ~CreateJob : 신규 직원 수~
- ~RetainedJob : 기존 직원 수~
- FranchiseCode	: 0=프랜차이즈 아님, 1=프랜차이즈
- UrbanRural : 0 = 도시, 1 = 농촌
- LowDoc : SBA 대출프로그램 신청 여부(0=아니오, 1 = 예)
- ~ChgOffDate : 채무 불이행으로 선언되는 날짜~
- DisbursementDate : 대금 지급일
- ~DisbursementGross : 지급된 금액~
- `MIS_Status` : 채무불이행 여부 (0=채무이행, 1=채무불이행)
- ~ChgOffPrinGr : 청구된 금액~
- GrAppv : 은행에서 승인한 대출 금액
- SBA_Appv : SBA의 승인된 대출 보증 금액
- SameState : 기업이 위치한 State과 은행이 위치한 State이 동일한지 여부(0=동일하지 않음, 1=동일함)
- Recession : SBA 약정 발행 날짜가 글로벌 금융위기 시기에 포함되는지 여부(0=포함되지 않음, 1=포함)
- ChgOffDate2 : 채무 불이행으로 선언되는 날짜(결측치 대체)
- Term_re : 대출 기간 범주형 변수 (0=5년 미만, 1=5년 이상 15년 미만, 2=15년 이상)
- smallmedium : 0=500명 미만(소기업), 1=500명 이상 1500명 이하(중기업)
- createjob2 : 신규 직원 수 파생변수 0=0~3인, 1=4~20인, 2=21인 이상
- retainedjob2 : 기존 직원 수 파생변수 0=0~3인, 1=4~20인, 2=21인 이상
- dis_re : 0=1천만원 미만, 1=1천만원 이상 ~ 5천만원 미만, 2=5천만원 이상 ~ 1억 미만, 3=1억 이상 ~ 5억 미만, 4=5억 이상 ~ 10억 미만, 5=10억 이상
- appv_pct : SBA_Appv / GrAppv
- ~Year : 대금 지급일의 연도~
- ~Month : 대금 지급일의 달~
- ~DATE : 금리 데이터 날짜~
- DPRIME : 금리 데이터 대출 금리
- dprime2 : 0=저금리, 1=고금리(10% 이상)
- ~Density : 산업 밀도~
- ~Upper_Threshold : 성장 가능성 상~
- ~Lower_Threshold : 성장 가능성 하~
- ~Label : 성장 가능성 지표(1,2,3)~
- growth_potential : 성장 가능성

## 라이브러리 불러오기

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

%config InlineBackend.figure_format = 'retina'  #선명하게 만들기

from sklearn.model_selection import train_test_split

In [6]:
# Window의 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

## 데이터 불러오기
EDA 및 전처리가 완료된 df_final3.csv 파일 불러옴

In [7]:
data = pd.read_csv('df_final3.csv')
df = data.copy() #원본 데이터

In [8]:
print(df.shape)
display(df.head())
display(df.tail())
print(df.info())

(864747, 44)


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,Year,Month,DATE,DPRIME,dprime2,Density,Upper_Threshold,Lower_Threshold,Label,growth_potential
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,17,1997-02-28,1997,...,1999,2,1999-02-01,7.75,0,0.172105,0.118196,0.055178,1,1
1,1000935004,Hairline Inc.,Vancouver,WA,98663,WELLS FARGO BANK NATL ASSOC,SD,14,1997-04-08,1997,...,1999,2,1999-02-01,7.75,0,0.090095,0.134566,0.055910,3,2
2,1003535008,Jason A. Philbrook,Rockland,ME,4841,KEYBANK NATIONAL ASSOCIATION,OH,18,1999-01-11,1999,...,1999,2,1999-02-01,7.75,0,0.041237,0.213874,0.051896,5,2
3,1009994000,"ARROW ENTERPRISE, INC.",BILLINGS,MT,59101,WESTERN SECURITY BANK,MT,5,1997-03-04,1997,...,1999,2,1999-02-01,7.75,0,0.148380,0.209033,0.057460,3,1
4,1011904005,"ROBIN E. JONES, D.D.S.",TACOMA,WA,98404,KEYBANK NATIONAL ASSOCIATION,OR,8,1997-03-05,1997,...,1999,2,1999-02-01,7.75,0,0.094218,0.134566,0.055910,3,2


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,Year,Month,DATE,DPRIME,dprime2,Density,Upper_Threshold,Lower_Threshold,Label,growth_potential
864742,9022713002,"BJORK, LAURA BEAUTY CONSULTANT",DALE CITY,VA,22193,MANUFACTURERS & TRADERS TR CO,MD,1,1996-02-21,1996,...,1996,2,1996-02-01,8.25,0,0.104331,0.139452,0.056099,3,1
864743,9035193000,"PROFESSIONALS FOR NONPROFITS,I",NEW YORK,NY,10022,BANCO POPULAR NORTH AMERICA,NY,10,1996-02-26,1996,...,1996,2,1996-02-01,8.25,0,0.094630,0.103363,0.066158,3,1
864744,9256453010,NORTHAIRE MARKET INC,SHAKOPEE,MN,55379,FIDELITY BANK,MN,20,1996-05-07,1996,...,1996,2,1996-02-01,8.25,0,0.066200,0.133574,0.063179,3,2
864745,9459933001,COMPUTER RENAISSANCE,CARY,NC,27511,FIRST-CITIZENS BK & TR CO,NC,17,1996-08-15,1996,...,1996,2,1996-02-01,8.25,0,0.187560,0.111641,0.060808,1,1
864746,7377094002,NORTHLAND DATACOMM CO,MINNEAPOLIS,MN,55311,BMO HARRIS BK NATL ASSOC,IL,20,2004-04-22,2004,...,2014,12,2014-12-01,3.25,0,0.066200,0.133574,0.063179,3,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864747 entries, 0 to 864746
Data columns (total 44 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      864747 non-null  int64  
 1   Name               864747 non-null  object 
 2   City               864747 non-null  object 
 3   State              864747 non-null  object 
 4   Zip                864747 non-null  int64  
 5   Bank               864747 non-null  object 
 6   BankState          864747 non-null  object 
 7   NAICS              864747 non-null  int64  
 8   ApprovalDate       864747 non-null  object 
 9   ApprovalFY         864747 non-null  int64  
 10  Term               864747 non-null  int64  
 11  NoEmp              864747 non-null  int64  
 12  NewExist           864747 non-null  int64  
 13  CreateJob          864747 non-null  int64  
 14  RetainedJob        864747 non-null  int64  
 15  FranchiseCode      864747 non-null  int64  
 16  Ur

In [19]:
# 랜덤 시드 설정
random_seed = 42

# train과 test 데이터로 분할
train_df, test_df = train_test_split(df, train_size=270000, test_size=30000, random_state=random_seed)

print("Train 데이터셋 크기:", len(train_df))
print("Test 데이터셋 크기:", len(test_df))

Train 데이터셋 크기: 270000
Test 데이터셋 크기: 30000


In [20]:
train_df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,Year,Month,DATE,DPRIME,dprime2,Density,Upper_Threshold,Lower_Threshold,Label,growth_potential
203979,1932945007,"Adom Restaurant, LLC",Fairfield,OH,45014,UMPQUA BANK,OR,1,2006-07-21,2006,...,2006,7,2006-07-01,8.25,0,0.109288,0.117011,0.057590,3,1
822925,8646673007,TEMPO LITHOGRAPH INC.,SANTA CLARA,CA,95054,WELLS FARGO BANK NATL ASSOC,CA,14,1995-09-08,1995,...,1996,1,1996-01-01,8.50,0,0.106632,0.111715,0.076095,3,2
190262,4415154005,A CLOSER LOOK AT STUDY SKILLS,MANCHESTER,NH,3103,CITIZENS BANK NATL ASSOC,NH,6,2001-04-03,2001,...,2001,4,2001-04-01,7.81,0,0.011745,0.201021,0.055319,5,3
261266,7549494001,NANO-WHITE CORPORATION,LIVERMORE,CA,94551,CAPITAL ONE NATL ASSOC,VA,10,2004-06-23,2004,...,2004,10,2004-10-01,4.75,0,0.111715,0.111715,0.076095,3,1
516156,7800654002,JULIE A HODGE M D A CALIFORNIA,FULLERTON,CA,92835,OPUS BANK,CA,8,2004-09-14,2004,...,2004,12,2004-12-01,5.15,0,0.076095,0.111715,0.076095,3,2


In [21]:
test_df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,Year,Month,DATE,DPRIME,dprime2,Density,Upper_Threshold,Lower_Threshold,Label,growth_potential
825107,8755153003,"DAUGHTERS, SAN CLEMENTE, INC.",SAN CLEMENTE,CA,92673,CITIZENS BUSINESS BANK,CA,13,1995-10-31,1996,...,1996,1,1996-01-01,8.50,0,0.098523,0.111715,0.076095,3,2
212849,5270194009,SANCHEZ MEDICAL CLINIC,BELLFLOWER,CA,90280,ALTAPACIFIC BANK,CA,8,2002-03-13,2002,...,2002,3,2002-03-01,4.75,0,0.076095,0.111715,0.076095,3,2
724805,4825853010,CALHOUN FARM,MENDENHALL,MS,39114,FIRST FINANCIAL BANK,AR,3,1992-04-29,1992,...,1993,4,1993-04-01,6.00,0,0.363477,0.363477,0.046565,3,2
525131,2156035006,"LUCHIANO'S PIZZERIA, INC",WAITE PARK,MN,56387,STEARNS BK NATL ASSOC,MN,1,2006-12-01,2007,...,2006,12,2006-12-01,8.25,0,0.103453,0.133574,0.063179,3,1
256633,3412594008,CHIPPERY,AUSTIN,TX,78744,COMPASS BANK,AL,10,1999-11-24,2000,...,2000,1,2000-01-01,8.50,0,0.087442,0.127318,0.069924,3,1


In [22]:
# train, validation 데이터로 분할
train, valid = train_test_split(train_df, train_size=210000, test_size=54747, random_state=random_seed)

print("Train 데이터셋 크기:", len(train))
print("Validation 데이터셋 크기:", len(valid))

Train 데이터셋 크기: 210000
Validation 데이터셋 크기: 54747


In [23]:
train.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,Year,Month,DATE,DPRIME,dprime2,Density,Upper_Threshold,Lower_Threshold,Label,growth_potential
415527,7718584006,MOTIONADAZ LLC,COCONUT CREEK,FL,33073,NEWTEK SMALL BUS. FINANCE INC.,NY,14,2004-08-20,2004,...,2004,9,2004-09-01,4.60,0,0.104862,0.104862,0.071389,3,2
742904,8362463001,ARMENDARIZ TAX & FINANCIAL SVC,LOS ANGELES,CA,90022,BANK OF HAWAII,HI,14,1995-05-23,1995,...,1995,7,1995-07-01,8.79,0,0.106632,0.111715,0.076095,3,2
94505,1908396004,CHARLEY'S GRILLED SUBS,ANTIOCH,CA,94509,COMERICA BANK,TX,1,2006-05-02,2006,...,2006,8,2006-08-01,8.25,0,0.088018,0.111715,0.076095,3,1
671996,3099743001,NEWELL MECHANICAL INC,RANCHO CORDOVA,CA,95670,SBA - EDF ENFORCEMENT ACTION,CO,6,1988-01-05,1988,...,1988,2,1988-02-01,8.51,0,0.009764,0.111715,0.076095,5,3
349923,6162254000,DOUBLE TT DINER AND TROOPER DI,BLUE BELL,PA,19422,BANK OF AMERICA NATL ASSOC,NC,1,2003-03-11,2003,...,2003,3,2003-03-01,4.25,0,0.114305,0.114305,0.051954,3,1


In [24]:
valid.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,Year,Month,DATE,DPRIME,dprime2,Density,Upper_Threshold,Lower_Threshold,Label,growth_potential
505979,3169045002,Just Soccer Inc.,West Warwick,RI,2893,WASHINGTON TR CO OF WESTERLY,RI,17,2008-07-28,2008,...,2008,8,2008-08-01,5.00,0,0.176755,0.125562,0.056036,1,1
177165,2746454007,CACTUS CLUB,LAS VEGAS,NV,89130,NEVADA STATE BANK,NV,20,1999-01-28,1999,...,1999,3,1999-03-01,7.75,0,0.059172,0.122341,0.059172,3,2
473911,2727645006,MOXIE'S RESTAURANT & LOUNGE,LITITZ,PA,17543,"TD BANK, NATIONAL ASSOCIATION",DE,1,2007-09-27,2007,...,2008,1,2008-01-01,6.92,0,0.114305,0.114305,0.051954,3,1
266038,1949285006,"MORTGAGE ABILITY, INC.",AIEA,HI,96701,CAPITAL ONE NATL ASSOC,VA,7,2006-08-02,2006,...,2006,9,2006-09-01,8.25,0,0.007246,0.115942,0.061159,5,2
173326,2997704004,"CONCRETE CONSTRUCTION SUPPLY,I",MERIDIAN,ID,83642,IDAHO BANKING COMPANY,ID,20,1999-06-09,1999,...,1999,8,1999-08-01,8.06,0,0.057310,0.155919,0.057310,3,2
